In [4]:
%pip install git+https://github.com/facebookresearch/audiocraft.git


  Cloning https://github.com/facebookresearch/audiocraft.git to /tmp/pip-req-build-4turu2fi
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft.git /tmp/pip-req-build-4turu2fi
  Resolved https://github.com/facebookresearch/audiocraft.git to commit 896ec7c47f5e5d1e5aa1e4b260c4405328bf009d
  Preparing metadata (setup.py) ... done
  Using cached av-11.0.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached flashy-0.0.2.tar.gz (72 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached hydra_colorlog-1.2.0-py3-none-any.whl.metadata (949 bytes)
  Using cached julius-0.2.7.tar.gz (59 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of audiocraft to determine which version is compatible with other require

In [ ]:

from audiocraft.models import MusicGen
import torchaudio
from IPython.display import Audio, display
import numpy as np
import time, random
import json

# Load MusicGen
model = MusicGen.get_pretrained('facebook/musicgen-small')

# Genre classifier (keeping your original system)
genre_keywords = {
    "nature": ["forest", "birds", "trees", "wind", "jungle"],
    "rain": ["rain", "storm", "droplets", "waterfall"],
    "calm": ["calm", "relax", "peace", "slow", "quiet"],
    "flute": ["flute", "bansuri", "indian", "instrumental"],
    "zen": ["zen", "meditation", "bowl", "lofi"],
    "binaural": ["binaural", "focus", "concentration", "deep"]
}

genre_prompt_map = {
    "nature": "relaxing ambient music with birds and forest sounds",
    "rain": "soothing rain sounds with soft ambient background",
    "calm": "peaceful meditation tones with gentle pads",
    "flute": "soothing Indian flute with gentle background pads",
    "zen": "lofi zen meditation beats with Tibetan bowls and rain",
    "binaural": "deep meditation music with binaural beats and ambient tones"
}

def classify_genre(prompt):
    prompt = prompt.lower()
    for genre, keywords in genre_keywords.items():
        if any(word in prompt for word in keywords):
            return genre
    return "calm"

def json_to_prompt(json_data):
    """
    LLM-like function to convert JSON input to music generation prompt
    """
    # Extract key information from JSON
    mood = json_data.get("currentMood", "calm")
    energy = json_data.get("energyLevel", [5])[0] if json_data.get("energyLevel") else 5
    stress = json_data.get("stressLevel", [5])[0] if json_data.get("stressLevel") else 5
    goals = json_data.get("selectedGoals", ["focus"])
    binaural_freq = json_data.get("binauralFrequency", "")
    ambient_sounds = json_data.get("ambientSounds", [])
    personal_intention = json_data.get("personalIntention", "")
    include_guidance = json_data.get("includeGuidance", False)
    include_breathing = json_data.get("includeBreathing", False)

    # Build descriptive prompt based on JSON data
    prompt_parts = []

    # Add mood-based descriptions
    if mood == "tired":
        prompt_parts.append("relaxing sleep-inducing")
    elif mood == "stressed":
        prompt_parts.append("calming stress-relief")
    elif mood == "anxious":
        prompt_parts.append("peaceful anxiety-reducing")
    elif mood == "energetic":
        prompt_parts.append("uplifting energizing")
    else:
        prompt_parts.append("peaceful meditation")

    # Add energy level considerations
    if energy <= 3:
        prompt_parts.append("gentle low-energy")
    elif energy >= 7:
        prompt_parts.append("vibrant high-energy")
    else:
        prompt_parts.append("balanced moderate")

    # Add goals
    if "focus" in goals:
        prompt_parts.append("concentration-enhancing")
    if "sleep" in goals:
        prompt_parts.append("sleep-inducing")
    if "relaxation" in goals:
        prompt_parts.append("deeply relaxing")

    # Add binaural frequency info
    if binaural_freq:
        if binaural_freq == "delta":
            prompt_parts.append("deep sleep delta waves")
        elif binaural_freq == "theta":
            prompt_parts.append("meditation theta waves")
        elif binaural_freq == "alpha":
            prompt_parts.append("relaxed alpha waves")
        elif binaural_freq == "beta":
            prompt_parts.append("focused beta waves")
        else:
            prompt_parts.append("binaural beats")

    # Add ambient sounds
    for sound in ambient_sounds:
        if sound == "forest":
            prompt_parts.append("forest birds nature")
        elif sound == "rain":
            prompt_parts.append("gentle rain")
        elif sound == "ocean":
            prompt_parts.append("ocean waves")
        elif sound == "wind":
            prompt_parts.append("soft wind")

    # Add personal intention
    if personal_intention:
        prompt_parts.append(personal_intention)

    # Add breathing/guidance elements
    if include_breathing:
        prompt_parts.append("breathing meditation")
    if include_guidance:
        prompt_parts.append("guided meditation")

    # Combine all parts into a coherent prompt
    full_prompt = " ".join(prompt_parts) + " music"

    return full_prompt

# Collect user input and convert to JSON
print("🎵 Welcome to AI Music Generator!")
print("Let's create your personalized meditation music...")
print("-" * 50)

# Ask user questions
print("How are you feeling right now?")
print("1. Tired 😴  2. Stressed 😰  3. Anxious 😟  4. Energetic ⚡  5. Calm 😌")
mood_choice = input("Enter number (1-5): ")
mood_map = {"1": "tired", "2": "stressed", "3": "anxious", "4": "energetic", "5": "calm"}
current_mood = mood_map.get(mood_choice, "calm")

energy_level = int(input("\n⚡ Energy Level (1-10, where 1=very low, 10=very high): "))
stress_level = int(input("😤 Stress Level (1-10, where 1=no stress, 10=very stressed): "))

print("\n🎯 What's your main goal? (you can choose multiple)")
print("1. Focus  2. Sleep  3. Relaxation  4. Meditation")
goals_input = input("Enter numbers separated by commas (e.g., 1,3): ")
goals_map = {"1": "focus", "2": "sleep", "3": "relaxation", "4": "meditation"}
selected_goals = [goals_map[g.strip()] for g in goals_input.split(",") if g.strip() in goals_map]

print("\n🎵 Which genre would you like to meditate with?")
print("1. Nature/Forest  2. Rain  3. Calm/Peaceful  4. Flute  5. Zen/Lofi  6. Binaural")
genre_choice = input("Enter number (1-6): ")
genre_map = {"1": "nature", "2": "rain", "3": "calm", "4": "flute", "5": "zen", "6": "binaural"}
chosen_genre = genre_map.get(genre_choice, "calm")

# Map genre to ambient sounds and binaural frequency
ambient_map = {
    "nature": ["forest"], "rain": ["rain"], "calm": [],
    "flute": [], "zen": ["forest"], "binaural": []
}
binaural_map = {
    "nature": "", "rain": "", "calm": "alpha",
    "flute": "theta", "zen": "theta", "binaural": "delta"
}

minutes = int(input("\n⏳ How long do you want the music (in minutes)? "))


# Create JSON from user inputs
json_data = {
    "currentMood": current_mood,
    "energyLevel": [energy_level],
    "stressLevel": [stress_level],
    "selectedGoals": selected_goals if selected_goals else ["focus"],
    "duration": [minutes],
    "binauralFrequency": binaural_map[chosen_genre],
    "ambientSounds": ambient_map[chosen_genre],
    "volume": [55],
    "includeGuidance": False,
    "includeBreathing": True,
    "personalIntention": personal_intention,
    "reminderTime": "",
    "sessionName": "custom",
    "generatedAt": time.strftime("%Y-%m-%dT%H:%M:%S.000Z"),
    "sessionId": f"mood_{int(time.time()*1000)}"
}

print("\n✅ Created your personalized session JSON!")
print("🔄 Converting to music prompt...")

# Convert JSON to prompt using our LLM-like function
generated_prompt = json_to_prompt(json_data)
print(f"🤖 Generated prompt: {generated_prompt}")

# Use the user's chosen genre directly instead of classifying
genre = chosen_genre  # Use the genre the user selected
music_prompt = genre_prompt_map[genre]
print(f"🎼 Using your selected '{genre}' genre")
print(f"🎵 Final music prompt: {music_prompt}")

# Get duration from JSON
minutes = json_data.get("duration", [10])[0] if json_data.get("duration") else 10
print(f"⏳ Generating music for {minutes} minutes")

# Generate 15s clip
model.set_generation_params(duration=15)
clip = model.generate([music_prompt])

# Loop audio to match duration
total_seconds = minutes * 60
repeats = total_seconds // 15 + 1
looped_audio = clip[0].repeat(1, repeats)
final_audio = looped_audio[:, :total_seconds * 32000]

# Save & play
torchaudio.save("meditation_music.wav", final_audio.cpu(), sample_rate=32000)
display(Audio("meditation_music.wav"))

print("✅ Music generation complete!")

In [3]:
import os
import json
import time
import numpy as np
import torch
import torchaudio
from pathlib import Path
from typing import Dict, List, Optional, Union
import warnings
warnings.filterwarnings("ignore")

class MeditationMusicGenerator:
    """
    Enhanced meditation music generator with multiple model support
    """
    
    def __init__(self, model_type: str = "musicgen", device: str = "auto"):
        """
        Initialize the music generator
        
        Args:
            model_type: Type of model to use ('musicgen', 'bark', 'audioldm', 'stable_audio')
            device: Device to use ('cpu', 'cuda', 'auto')
        """
        self.model_type = model_type.lower()
        self.device = self._get_device(device)
        self.model = None
        self.sample_rate = 32000
        
        # Genre definitions with enhanced prompting
        self.genre_keywords = {
            "nature": ["forest", "birds", "trees", "wind", "jungle", "wilderness", "outdoor"],
            "rain": ["rain", "storm", "droplets", "waterfall", "precipitation", "drizzle"],
            "calm": ["calm", "relax", "peace", "slow", "quiet", "serene", "tranquil"],
            "flute": ["flute", "bansuri", "indian", "instrumental", "wind instrument"],
            "zen": ["zen", "meditation", "bowl", "lofi", "spiritual", "mindful"],
            "binaural": ["binaural", "focus", "concentration", "deep", "brainwave"],
            "ambient": ["ambient", "atmospheric", "spacial", "ethereal", "dreamy"],
            "classical": ["classical", "piano", "strings", "orchestral", "baroque"]
        }
        
        self.genre_prompt_map = {
            "nature": "peaceful ambient nature sounds with gentle forest atmosphere, birds chirping softly, leaves rustling in the wind",
            "rain": "soothing rain soundscape with gentle precipitation, soft thunder in distance, calming water drops",
            "calm": "serene meditation music with soft harmonic pads, gentle sustained tones, peaceful ambience",
            "flute": "meditative Indian flute melody with warm background drones, spiritual bansuri sounds, gentle wind instrument",
            "zen": "zen meditation music with Tibetan singing bowls, soft ambient pads, mindful lofi beats, spiritual atmosphere",
            "binaural": "deep meditation soundscape with binaural beats, ambient drones, consciousness-altering frequencies",
            "ambient": "ethereal ambient music with spacial reverb, dreamy atmospheric pads, floating soundscape",
            "classical": "gentle classical meditation with soft piano, light strings, peaceful orchestral arrangement"
        }
        
        self.mood_descriptors = {
            "tired": "sleep-inducing, deeply relaxing, slow tempo, gentle fade",
            "stressed": "stress-relief, anxiety-reducing, calming harmonies, tension release",
            "anxious": "anxiety-soothing, grounding, stable rhythms, reassuring tones",
            "energetic": "uplifting yet grounding, balanced energy, motivating but peaceful",
            "calm": "peaceful, balanced, centered, harmonious",
            "focused": "concentration-enhancing, clear minded, alert yet relaxed",
            "sad": "emotionally healing, gentle comfort, warm embracing tones",
            "happy": "joyful serenity, light hearted peace, uplifting calm"
        }
        
        self.binaural_frequencies = {
            "delta": "0.5-4Hz deep sleep delta brainwave frequencies",
            "theta": "4-8Hz meditation theta wave frequencies for deep relaxation",
            "alpha": "8-13Hz relaxed alpha wave frequencies for calm awareness",
            "beta": "13-30Hz focused beta wave frequencies for concentration",
            "gamma": "30-100Hz gamma wave frequencies for heightened awareness"
        }
        
        self._load_model()
    
    def _get_device(self, device: str) -> str:
        """Determine the best device to use"""
        if device == "auto":
            return "cuda" if torch.cuda.is_available() else "cpu"
        return device
    
    def _load_model(self):
        """Load the specified model"""
        try:
            if self.model_type == "musicgen":
                from audiocraft.models import MusicGen
                print(f"Loading MusicGen model on {self.device}...")
                self.model = MusicGen.get_pretrained('facebook/musicgen-small')
                if self.device == "cuda":
                    self.model = self.model.cuda()
                    
            elif self.model_type == "bark":
                from bark import SAMPLE_RATE, generate_audio, preload_models
                print(f"Loading Bark model on {self.device}...")
                preload_models()
                self.sample_rate = SAMPLE_RATE
                self.model = "bark_loaded"  # Bark uses global functions
                
            elif self.model_type == "audioldm":
                from diffusers import AudioLDMPipeline
                print(f"Loading AudioLDM model on {self.device}...")
                self.model = AudioLDMPipeline.from_pretrained(
                    "cvssp/audioldm-s-full-v2", 
                    torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
                )
                if self.device == "cuda":
                    self.model = self.model.to("cuda")
                self.sample_rate = 16000
                
            elif self.model_type == "stable_audio":
                try:
                    from stable_audio_tools import get_pretrained_model
                    print(f"Loading Stable Audio model on {self.device}...")
                    self.model, self.model_config = get_pretrained_model("stabilityai/stable-audio-open-1.0")
                    if self.device == "cuda":
                        self.model = self.model.to("cuda")
                    self.sample_rate = self.model_config["sample_rate"]
                except ImportError:
                    print("Stable Audio not available, falling back to MusicGen")
                    self.model_type = "musicgen"
                    self._load_model()
                    return
                    
            else:
                raise ValueError(f"Unsupported model type: {self.model_type}")
                
            print(f"✅ {self.model_type.upper()} model loaded successfully!")
            
        except Exception as e:
            print(f"❌ Error loading {self.model_type}: {e}")
            if self.model_type != "musicgen":
                print("Falling back to MusicGen...")
                self.model_type = "musicgen"
                self._load_model()
            else:
                raise e
    
    def classify_genre(self, prompt: str) -> str:
        """Classify genre from text prompt"""
        prompt_lower = prompt.lower()
        
        # Count matches for each genre
        genre_scores = {}
        for genre, keywords in self.genre_keywords.items():
            score = sum(1 for keyword in keywords if keyword in prompt_lower)
            if score > 0:
                genre_scores[genre] = score
        
        # Return genre with highest score, default to 'calm'
        if genre_scores:
            return max(genre_scores.items(), key=lambda x: x[1])[0]
        return "calm"
    
    def json_to_prompt(self, json_data: Dict) -> str:
        """
        Convert JSON meditation session data to music generation prompt
        """
        prompt_parts = []
        
        # Extract data with defaults
        mood = json_data.get("currentMood", "calm").lower()
        energy = json_data.get("energyLevel", [5])[0] if json_data.get("energyLevel") else 5
        stress = json_data.get("stressLevel", [5])[0] if json_data.get("stressLevel") else 5
        goals = json_data.get("selectedGoals", ["focus"])
        binaural_freq = json_data.get("binauralFrequency", "")
        ambient_sounds = json_data.get("ambientSounds", [])
        personal_intention = json_data.get("personalIntention", "")
        include_guidance = json_data.get("includeGuidance", False)
        include_breathing = json_data.get("includeBreathing", False)
        volume = json_data.get("volume", [55])[0] if json_data.get("volume") else 55
        
        # Add mood descriptors
        if mood in self.mood_descriptors:
            prompt_parts.append(self.mood_descriptors[mood])
        
        # Add energy level considerations
        if energy <= 3:
            prompt_parts.append("very gentle, minimal energy, soft dynamics")
        elif energy <= 5:
            prompt_parts.append("low to moderate energy, calm progression")
        elif energy <= 7:
            prompt_parts.append("balanced energy, moderate dynamics")
        else:
            prompt_parts.append("higher energy but still peaceful, uplifting")
        
        # Add stress level considerations
        if stress >= 7:
            prompt_parts.append("deeply calming, tension-releasing, therapeutic")
        elif stress >= 5:
            prompt_parts.append("stress-reducing, soothing, comforting")
        
        # Add goals
        goal_descriptors = {
            "focus": "concentration-enhancing, mental clarity, alert relaxation",
            "sleep": "sleep-inducing, deeply relaxing, drowsy, fade to silence",
            "relaxation": "deeply relaxing, tension release, peaceful rest",
            "meditation": "meditative, mindful, spiritual, consciousness-expanding",
            "healing": "therapeutic, healing frequencies, restorative",
            "creativity": "inspiring, creative flow, imaginative, innovative"
        }
        
        for goal in goals:
            if goal in goal_descriptors:
                prompt_parts.append(goal_descriptors[goal])
        
        # Add binaural frequency info
        if binaural_freq and binaural_freq in self.binaural_frequencies:
            prompt_parts.append(self.binaural_frequencies[binaural_freq])
        
        # Add ambient sounds
        ambient_descriptors = {
            "forest": "forest atmosphere, birds chirping, leaves rustling",
            "rain": "gentle rain sounds, water droplets, soft precipitation",
            "ocean": "ocean waves, seashore sounds, rhythmic water",
            "wind": "soft wind sounds, gentle breeze, air movement",
            "fire": "crackling fireplace, warm fire sounds, cozy flames",
            "bells": "meditation bells, chimes, singing bowls",
            "white_noise": "gentle white noise, consistent background"
        }
        
        for sound in ambient_sounds:
            if sound in ambient_descriptors:
                prompt_parts.append(ambient_descriptors[sound])
        
        # Add personal intention
        if personal_intention:
            prompt_parts.append(personal_intention)
        
        # Add breathing/guidance elements
        if include_breathing:
            prompt_parts.append("breathing rhythm, respiratory guidance, breath synchronization")
        if include_guidance:
            prompt_parts.append("guided meditation elements, gentle instruction tones")
        
        # Add volume considerations
        if volume <= 30:
            prompt_parts.append("very quiet, whisper-soft, barely audible")
        elif volume <= 50:
            prompt_parts.append("soft volume, gentle dynamics")
        elif volume >= 80:
            prompt_parts.append("fuller volume, rich presence")
        
        # Combine all parts
        full_prompt = ", ".join(prompt_parts)
        
        # Add musical structure
        full_prompt += ", ambient meditation music, peaceful composition, smooth transitions, no jarring sounds, continuous flow"
        
        return full_prompt
    
    def generate_audio_clip(self, prompt: str, duration: float = 15.0) -> torch.Tensor:
        """Generate a single audio clip"""
        try:
            if self.model_type == "musicgen":
                self.model.set_generation_params(duration=duration)
                with torch.no_grad():
                    audio = self.model.generate([prompt])
                return audio[0]
                
            elif self.model_type == "bark":
                from bark import generate_audio
                # Bark generates from text prompts differently
                audio_array = generate_audio(f"[ambient_music] {prompt}")
                return torch.tensor(audio_array).unsqueeze(0)
                
            elif self.model_type == "audioldm":
                with torch.no_grad():
                    audio = self.model(prompt, num_inference_steps=20, audio_length_in_s=duration)
                return torch.tensor(audio.audios[0]).unsqueeze(0)
                
            elif self.model_type == "stable_audio":
                from stable_audio_tools import get_pretrained_model
                conditioning = {
                    "prompt": prompt,
                    "seconds_start": 0,
                    "seconds_total": duration
                }
                with torch.no_grad():
                    audio = self.model.generate(conditioning)
                return audio
                
        except Exception as e:
            print(f"Error generating audio: {e}")
            # Return silence as fallback
            return torch.zeros(1, int(duration * self.sample_rate))
    
    def extend_audio(self, audio: torch.Tensor, target_duration: float) -> torch.Tensor:
        """Extend audio to target duration using intelligent looping"""
        current_length = audio.shape[-1]
        target_length = int(target_duration * self.sample_rate)
        
        if current_length >= target_length:
            return audio[:, :target_length]
        
        # Calculate how many complete loops we need
        num_complete_loops = target_length // current_length
        remaining_samples = target_length % current_length
        
        # Create the extended audio
        loops = [audio] * num_complete_loops
        if remaining_samples > 0:
            loops.append(audio[:, :remaining_samples])
        
        extended_audio = torch.cat(loops, dim=1)
        
        # Apply subtle fade between loops to avoid clicks
        fade_samples = min(1000, current_length // 20)  # 50ms fade or 5% of clip length
        
        for i in range(1, num_complete_loops + (1 if remaining_samples > 0 else 0)):
            start_idx = i * current_length
            end_idx = min(start_idx + fade_samples, extended_audio.shape[-1])
            
            if end_idx > start_idx:
                fade_in = torch.linspace(0, 1, end_idx - start_idx).to(extended_audio.device)
                prev_end = max(start_idx - fade_samples, 0)
                fade_out = torch.linspace(1, 0, start_idx - prev_end).to(extended_audio.device)
                
                if prev_end < start_idx:
                    extended_audio[:, prev_end:start_idx] *= fade_out
                extended_audio[:, start_idx:end_idx] *= fade_in
        
        return extended_audio
    
    def generate_meditation_music(self, json_input: Union[str, Dict], output_path: str = "meditation_music.mp4") -> str:
        """
        Main function to generate meditation music from JSON input
        
        Args:
            json_input: JSON string or dictionary with meditation parameters
            output_path: Output file path (supports .mp3, .wav, .mp4)
            
        Returns:
            Path to generated audio file
        """
        try:
            # Parse JSON input
            if isinstance(json_input, str):
                try:
                    session_data = json.loads(json_input)
                except json.JSONDecodeError as e:
                    raise ValueError(f"Invalid JSON format: {e}")
            else:
                session_data = json_input
            
            print("🎵 Starting meditation music generation...")
            print(f"🤖 Using {self.model_type.upper()} model")
            
            # Convert JSON to prompt
            generated_prompt = self.json_to_prompt(session_data)
            print(f"🔤 Generated prompt: {generated_prompt[:100]}...")
            
            # Classify genre and get specific prompt
            genre = self.classify_genre(generated_prompt)
            music_prompt = self.genre_prompt_map.get(genre, self.genre_prompt_map["calm"])
            
            # Combine prompts for better results
            final_prompt = f"{music_prompt}, {generated_prompt}"
            print(f"🎼 Genre: {genre}")
            print(f"🎵 Final prompt: {final_prompt[:150]}...")
            
            # Get duration
            duration_minutes = session_data.get("duration", [10])[0] if session_data.get("duration") else 10
            total_seconds = duration_minutes * 60
            print(f"⏳ Generating {duration_minutes} minutes of music...")
            
            # Generate initial clip (15-30 seconds depending on model)
            clip_duration = min(30.0, total_seconds)  # Don't generate more than needed for short sessions
            print(f"🎶 Generating initial {clip_duration}s clip...")
            
            audio_clip = self.generate_audio_clip(final_prompt, clip_duration)
            
            # Extend to full duration if needed
            if total_seconds > clip_duration:
                print(f"🔄 Extending audio to {duration_minutes} minutes...")
                final_audio = self.extend_audio(audio_clip, total_seconds)
            else:
                final_audio = audio_clip[:, :int(total_seconds * self.sample_rate)]
            
            # Ensure output directory exists
            output_path = Path(output_path)
            output_path.parent.mkdir(parents=True, exist_ok=True)
            
            # Save audio file
            print(f"💾 Saving audio to {output_path}...")
            
            # Normalize audio
            final_audio = final_audio / final_audio.abs().max() * 0.8
            
            if output_path.suffix.lower() in ['.mp3', '.mp4']:
                # For MP3/MP4, save as WAV first then convert if ffmpeg available
                temp_wav = output_path.with_suffix('.wav')
                torchaudio.save(str(temp_wav), final_audio.cpu(), self.sample_rate)
                
                try:
                    import subprocess
                    # Convert to MP3/MP4 using ffmpeg
                    cmd = ['ffmpeg', '-i', str(temp_wav), '-codec:a', 'aac', '-b:a', '128k', str(output_path), '-y']
                    subprocess.run(cmd, check=True, capture_output=True)
                    temp_wav.unlink()  # Remove temporary WAV file
                except (subprocess.CalledProcessError, FileNotFoundError):
                    print("⚠️  ffmpeg not found, saving as WAV instead")
                    output_path = temp_wav
            else:
                # Save as WAV
                torchaudio.save(str(output_path), final_audio.cpu(), self.sample_rate)
            
            print(f"✅ Music generation complete! Saved to: {output_path}")
            return str(output_path)
            
        except Exception as e:
            print(f"❌ Error generating music: {e}")
            raise e

def create_sample_json():
    """Create a sample JSON for testing"""
    return {
        "currentMood": "stressed",
        "energyLevel": [4],
        "stressLevel": [8],
        "selectedGoals": ["relaxation", "focus"],
        "duration": [15],  # 15 minutes
        "binauralFrequency": "alpha",
        "ambientSounds": ["forest", "rain"],
        "volume": [60],
        "includeGuidance": False,
        "includeBreathing": True,
        "personalIntention": "deep relaxation and mental clarity",
        "reminderTime": "",
        "sessionName": "evening_relaxation",
        "generatedAt": time.strftime("%Y-%m-%dT%H:%M:%S.000Z"),
        "sessionId": f"session_{int(time.time()*1000)}"
    }

# Example usage
if __name__ == "__main__":
    # Initialize generator with different models
    # Options: "musicgen", "bark", "audioldm", "stable_audio"
    
    generator = MeditationMusicGenerator(model_type="musicgen", device="auto")
    
    # Create sample session data
    sample_json = create_sample_json()
    
    # Generate music
    try:
        output_file = generator.generate_meditation_music(
            json_input=sample_json,
            output_path="output/meditation_session.mp4"
        )
        print(f"🎉 Success! Generated: {output_file}")
        
    except Exception as e:
        print(f"💥 Generation failed: {e}")

# Alternative models installation commands:
"""

"""

❌ Error loading musicgen: No module named 'audiocraft'


ModuleNotFoundError: No module named 'audiocraft'

In [2]:
# For MusicGen (default):
%pip install audiocraft

# For Bark:
%pip install git+https://github.com/suno-ai/bark.git

# For AudioLDM:
%pip install diffusers transformers accelerate

# For Stable Audio:
%pip install stable-audio-tools

# For audio format conversion:
# Install ffmpeg: https://ffmpeg.org/download.html%

  Using cached audiocraft-1.3.0.tar.gz (635 kB)
  Preparing metadata (setup.py) ... done
  Using cached av-11.0.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached flashy-0.0.2.tar.gz (72 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached hydra_colorlog-1.2.0-py3-none-any.whl.metadata (949 bytes)
  Using cached julius-0.2.7.tar.gz (59 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of audiocraft to determine which version is compatible with other requirements. This could take a while.
  Using cached audiocraft-1.2.0.tar.gz (615 kB)
  Preparing metadata (setup.py) ... done
  Using cached audiocraft-1.1.0.tar.gz (610 kB)
  Preparing metadata (setup.py) ... done
  Using cached spacy-3.5.2.tar.gz (1.2 MB)
  Installing build dependencies ... er